In [1]:
import urllib2
import cookielib
from bs4 import BeautifulSoup
import html5lib
import pandas as pd

url = 'http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/muni/ratehistory.cfm?swis=012200'

In [2]:
cj = cookielib.CookieJar()
opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
baseurl = "http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/county/equasearch.cfm?swis=01"
regex = '<ul class="w462">(.*?)</ul>'

opener.open(baseurl)
urllib2.install_opener(opener)

request = urllib2.Request('http://orpts.tax.ny.gov/cfapps/MuniPro/muni_theme/muni/ratehistory.cfm?swis=010300')
request.add_header('Referer', baseurl)

requestData = urllib2.urlopen(request)
page = requestData.read()

In [3]:
starting_pos = page.find('<table summary="Equalization Rate History"')

soup = BeautifulSoup(page[starting_pos:])

In [14]:
eq_table = pd.read_html(page[starting_pos:], flavor='bs4', header=0)

for tr in soup.find_all('tr'):
    tds = tr.find_all('td')
    print "Nome: %s, Cognome: %s, Email: %s" % \
          (tds[0].text, tds[0].text, tds[0].text)

In [29]:
eq_table[0].iloc[:,:4].to_csv('010300.csv', index=False)